In [ ]:
# Import the necessary functions
from search_Croma import fetch_prices_from_croma
from search_reliance import fetch_prices_from_reliance
import json

# Define the product query
query = "iphone 16 pro max"

# Fetch prices from Croma
croma_prices = fetch_prices_from_croma(query)
print("Croma Prices:")
print(json.dumps(croma_prices, indent=4))

# Fetch prices from Reliance Digital
reliance_prices = fetch_prices_from_reliance(query)
print("Reliance Digital Prices:")
print(json.dumps(reliance_prices, indent=4))
